# Business Cycle - Currencies

## Imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Cleaning

In [2]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_207/711168680.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [3]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_207/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [4]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World
Date,,
1986-12-31,242.1700,NaN
1987-01-30,274.7800,NaN
1987-02-27,285.6377,NaN
1987-03-31,293.8792,NaN
1987-04-30,291.2698,NaN
...,...,...
2025-07-31,14412.5527,8214.1572
2025-08-29,14704.7217,8431.0801
2025-09-30,15240.0381,8705.7139


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [5]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

display(CPI_forecasts)

RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

display(RGDP_forecasts)

,UK,CH,JP,AU,EU,US,EM
Date,,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17,NaN
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79,NaN
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79,NaN


,AU,UK,CH,JP,EU,US,EM,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Year,RGDP,YoY Growth
Date,,,,,,,,,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN,NaN,NaN,NaN,NaN,196000.0,1.610231e+13,NaN
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN,NaN,NaN,NaN,NaN,196100.0,1.171121e+13,-27.27
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN,NaN,NaN,NaN,NaN,196200.0,1.105773e+13,-5.58
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250,NaN,NaN,NaN,NaN,NaN,196300.0,1.219667e+13,10.30
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250,NaN,NaN,NaN,NaN,NaN,196400.0,1.441403e+13,18.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# --- Load Currency Prices ---
currency_data = pd.read_excel(
    f"{path}/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
currency_data.index = pd.to_datetime(currency_data.index)
currency_data.index = currency_data.index + pd.offsets.MonthEnd(0)
currency_XR = currency_data

display(currency_XR)

,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


## Global Variables

In [7]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [8]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)

display(CPI_component)
display(RGDP_component)

# # Reindex RGDP trend to match CPI trend index exactly
# RGDP_component = RGDP_trend.reindex(CPI_trend.index).ffill()
# CPI_component = CPI_trend

,UK,CH,JP,AU,EU,US,EM
Date,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,1.31793,-1.22807,0.5,-1.71445,-0.54354,0.39,NaN
2025-07-31,1.17674,-1.09059,0.3,0.39146,-0.57075,0.39,NaN
2025-08-31,1.00881,-0.86037,-0.3,0.39146,-0.12434,0.39,NaN


,AU,UK,CH,JP,EU,US,EM,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Year,RGDP,YoY Growth
Date,,,,,,,,,,,,,,
1970-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-1.71445,0.7,2.356965,2.957250,0.993241,-0.529850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# --- Composite Signal Construction ---
# Logic: For Currencies, we want High Growth (+) and High(ish) Inflation (+)
# We combine them: 50% Growth, 50% Inverse Inflation
business_cyle_signal = (0.5 * RGDP_component) + (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal)

'Business Cycle Signal:'

,AU,CH,EM,EU,JP,RGDP,UK,US,Unnamed: 10,Unnamed: 11,Unnamed: 8,Unnamed: 9,Year,YoY Growth
Date,,,,,,,,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-1.71445,0.564448,NaN,0.224851,1.728625,NaN,1.008965,-0.069925,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-31,0.39146,-0.867678,NaN,-0.092139,0.488158,NaN,1.438370,0.026338,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-31,0.39146,-0.752568,NaN,0.131066,0.188158,NaN,1.354405,0.026338,NaN,NaN,NaN,NaN,NaN,NaN


## Asset Class Returns

In [10]:
currency_XR['US'] = 0.0
display(currency_XR)

,CH,EU,JP,UK,AU,EM,US
Date,,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN,0.0
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN,0.0
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN,0.0
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN,0.0
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN,0.0
...,...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021,0.0
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627,0.0
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796,0.0


## Portfolio Construction

In [11]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = business_cyle_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,AU,CH,EM,EU,JP,RGDP,UK,US,Unnamed: 10,Unnamed: 11,Unnamed: 8,Unnamed: 9,Year,YoY Growth
Date,,,,,,,,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,6.0,3.0,NaN,4.0,1.0,NaN,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-31,3.0,6.0,NaN,5.0,2.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-31,2.0,6.0,NaN,4.0,3.0,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_stds = rank.std(axis = 1)
standardized_weights = rank.sub(rank_mean, axis = 0).div(rank_stds, axis = 0)

display(standardized_weights)

,AU,CH,EM,EU,JP,RGDP,UK,US,Unnamed: 10,Unnamed: 11,Unnamed: 8,Unnamed: 9,Year,YoY Growth
Date,,,,,,,,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,1.336306,-0.267261,NaN,0.267261,-1.336306,NaN,-0.801784,0.801784,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-31,-0.267261,1.336306,NaN,0.801784,-0.801784,NaN,-1.336306,0.267261,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-31,-0.801784,1.336306,NaN,0.267261,-0.267261,NaN,-1.336306,0.801784,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * currency_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-01-31':'2025-10-31']

display(final_strategy_weights)

,AU,CH,EM,EU,JP,RGDP,UK,US,Unnamed: 10,Unnamed: 11,Unnamed: 8,Unnamed: 9,Year,YoY Growth
Date,,,,,,,,,,,,,,
1998-01-31,0.640155,-1.280311,0.0,-0.640155,1.280311,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1998-02-28,0.575363,-1.150726,0.0,-0.575363,1.150726,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1998-03-31,1.155085,-1.155085,0.0,-0.577543,0.577543,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1998-04-30,-0.568520,-1.137041,0.0,0.000000,1.137041,0.0,0.000000,0.568520,0.0,0.0,0.0,0.0,0.0,0.0
1998-05-31,-0.558162,-1.116323,0.0,0.000000,1.116323,0.0,0.000000,0.558162,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,1.160847,-0.232169,0.0,0.232169,-1.160847,0.0,-0.696508,0.696508,0.0,0.0,0.0,0.0,0.0,0.0
2025-07-31,-0.238570,1.192848,0.0,0.715709,-0.715709,0.0,-1.192848,0.238570,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-31,-0.698872,1.164787,0.0,0.232957,-0.232957,0.0,-1.164787,0.698872,0.0,0.0,0.0,0.0,0.0,0.0


## Backtest

In [14]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [15]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = currency_XR,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_XR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

KeyError: "['RGDP', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 8', 'Unnamed: 9', 'Year', 'YoY Growth'] not in index"

## Performance Statistics

In [ ]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,6.9887,3.9994
Arithm Avg Xs Return,4.9498,1.9604
Std Xs Returns,15.5470,10.5390
Sharpe Arithmetic,0.3184,0.1860
Geom Avg Total Return,5.9249,3.5072
Geom Avg Xs Return,3.8686,1.4509
Sharpe Geometric,0.2488,0.1377
Min Xs Return,-19.0940,-7.5150
Max Xs Return,12.8084,12.0313
Skewness,-0.6049,0.6222
